# FSHD Risk Analysis

This notebook is meant to read, parse, create family trees and get data for each patient from the main FSHD Italian Registry.
This is then used to train a network that can regress the probability that a new individual in the family tree shows sign of the disease given its existing family tree data.

In [ ]:
import pandas as pd
import numpy as np 
import networkx as nx
from matplotlib import pyplot as plt
import os
import re as re
import math as math
from tabulate import tabulate
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score,roc_auc_score,make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_curve


## Input datasets

This program reads in input family tree files exported in CSV format (comma separated values) by HaploPainter.
It also fixes some typical problems before parsing a new file.
By default, it searches for family trees in all subfolders of the `data` folder.


In [ ]:
def fix_family_file(fileName):
    lines = []
    file_content = []
    with open(fileName, "r") as f:
        lines = f.readlines()
    with open(fileName, "w") as f:
        for line in lines:
            if "PERSON" not in line:
                line = re.sub(r'(\d+)(\/\d{2})', lambda x: x.group(1).zfill(4)+x.group(2), line )
                f.write(line)
                

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)


In [ ]:
family_df = []

print("Searching for family trees in the data folder")
for root, dirs, files in os.walk("data/"):
    for file in files:
        if file.endswith(".utf8"):
            filePath = os.path.join(root, file)
            fix_family_file(filePath)
            family_df.append(pd.read_csv(filePath, header=None, usecols=[0,1,2,3], names=['FAMILY', 'PERSON','FATHER','MOTHER'], delimiter='\t', index_col=1))
            print("found family tree: ", filePath)
print(len(family_df)," family trees found")


The patients database is loaded here. Only the Excel Sheet `Tesi` is considered.

In [ ]:


# print("Now opening the patients database. This might take a while...")

# patients_df = pd.read_excel (r'data/dataset_tesi_finale_23012022.xlsx', sheet_name='Tesi', usecols=[0,1,5,6,7,8,9,10,13,18,38,39,40,685,686,687,688,689,690], index_col=0)
# patients_df.columns = patients_df.columns.str.replace(' ', '_')
# patients_df.columns = patients_df.columns.str.replace('_E2_C3_', '')
# patients_df.columns = patients_df.columns.str.replace('_CCFf', '')
# patients_df.dropna(subset=['FSHD_No_null','Ch4-1_no_null_validated','Date_of_Birth'], inplace=True)

# patients_df["FACIALWEAKNESS"] = pd.to_numeric(patients_df["FACIALWEAKNESS"])
# patients_df["SCAPGIRDLEINVOLV"] = pd.to_numeric(patients_df["SCAPGIRDLEINVOLV"])
# patients_df["UPPERLIMBSINVOLV"] = pd.to_numeric(patients_df["UPPERLIMBSINVOLV"])
# patients_df["LEGSINVOLVEMENT"] = pd.to_numeric(patients_df["LEGSINVOLVEMENT"])
# patients_df["PELVGIRDLEINVOLV"] = pd.to_numeric(patients_df["PELVGIRDLEINVOLV"])
# patients_df["ABDMUSCLEINVOLV"] = pd.to_numeric(patients_df["ABDMUSCLEINVOLV"])
# patients_df["Date_of_Birth"] = pd.to_datetime(patients_df["Date_of_Birth"])

# print("Database opened successfully.")
# patients_df['Classificazione_no_null'] = patients_df['Classificazione_no_null'].fillna("Z")
# patients_df.head()
# print("Data from ", len(patients_df.index), " patients loaded.")

# patients_df.columns = patients_df.columns.str.replace('Classificazione_no_null', 'clinical_category')
# patients_df.columns = patients_df.columns.str.replace('ID_famiglia', 'family_id')
# patients_df.columns = patients_df.columns.str.replace('Ch4-1_no_null_validated', 'allele_size_in_kb')
# patients_df.columns = patients_df.columns.str.replace('FSHD_No_null', 'FSHD_score')


# patients_df["Birth_Year"] = pd.to_numeric(patients_df["Birth_Year"])
# patients_df["age_at_evaluation"] = pd.to_numeric(patients_df["age_at_evaluation"])
# patients_df['age_at_evaluation'] = patients_df['age_at_evaluation'].fillna(-1)

# patients_df["Year_at_evaluation"] = pd.to_numeric(patients_df["Year_at_evaluation"])

# #print(patients_df.columns)
# patients_df.to_hdf('patients_data.h5', key='df', mode='w')
# #print(patients_df.dtypes)
patients_df = pd.read_hdf('patients_data.h5')
print(patients_df)


In [ ]:
def calculateDegreeOfKinship(dag, source, target):
    distance = -1
    # if source == target:
    #     return 0
    ancestor = nx.lowest_common_ancestor(dag,source, target, default=-1)
    if ancestor != -1:
        distance = nx.shortest_path_length(dag,ancestor,source) +nx.shortest_path_length(dag,ancestor,target)
    return distance


In [ ]:
ensure_dir("family_graphs/")
training_df = pd.DataFrame(columns=['gender','DRA', 'age_at_evaluation','allele_length',
                                    'severity', 'severity_binary', 'severity_from_score_binary', 'FSHD_score', 'category', 
                                    'max_score_1st_kinship', 'max_score_2nd_kinship', 'max_score_3rd_kinship', 'max_score_higher_kinship',
                                    'max_facial_partial_score_1st_kinship', 'max_facial_partial_score_2nd_kinship', 'max_facial_partial_score_3rd_kinship', 'max_facial_partial_score_higher_kinship',
                                    'max_scapular_girdle_partial_score_1st_kinship', 'max_scapular_girdle_partial_score_2nd_kinship', 'max_scapular_girdle_partial_score_3rd_kinship', 'max_scapular_girdle_partial_score_higher_kinship',
                                    'max_upper_limbs_partial_score_1st_kinship', 'max_upper_limbs_partial_score_2nd_kinship', 'max_upper_limbs_partial_score_3rd_kinship', 'max_upper_limbs_partial_score_higher_kinship',
                                    'max_legs_partial_score_1st_kinship', 'max_legs_partial_score_2nd_kinship', 'max_legs_partial_score_3rd_kinship', 'max_legs_partial_score_higher_kinship',
                                    'max_pelvic_girdle_partial_score_1st_kinship', 'max_pelvic_girdle_partial_score_2nd_kinship', 'max_pelvic_girdle_partial_score_3rd_kinship', 'max_pelvic_girdle_partial_score_higher_kinship',
                                    'max_abdomen_partial_score_1st_kinship', 'max_abdomen_partial_score_2nd_kinship', 'max_abdomen_partial_score_3rd_kinship', 'max_abdomen_partial_score_higher_kinship',])
numberOfFamilies = 0
maxIndividualsInFamily = 0
allele_length = {}

for df in family_df: 
    family = df[['FAMILY','FATHER','MOTHER']]
    familyName = ""
    personId=[]
    fshd_score ={}
    gender = {}
    DRA = {}
    clinical_category = {}
    partial_scores = {}
    age_at_evaluation = {}
    for index, row in family.iterrows():
        familyName = row['FAMILY']
        if '/' in index and index in patients_df.index :
            patients_row = patients_df.loc[index]
            tmp_score = patients_row["FSHD_score"]
            tmp_allele_length = patients_row["allele_size_in_kb"]
            tmp_clinical_category = patients_row["clinical_category"]
            tmp_partial_scores = [patients_row["FACIALWEAKNESS"],
                                  patients_row["SCAPGIRDLEINVOLV"],
                                  patients_row["UPPERLIMBSINVOLV"],
                                  patients_row["LEGSINVOLVEMENT"],
                                  patients_row["PELVGIRDLEINVOLV"],
                                  patients_row["ABDMUSCLEINVOLV"]]
            if isinstance(tmp_score,float) and not math.isnan(tmp_score) and not math.isnan(tmp_allele_length) and tmp_clinical_category != "Z":
                personId.append(index)
                fshd_score[index] = tmp_score
                gender[index] = patients_row["Sex"]
                DRA[index] = patients_row["DRA_(0=no;1=si)"]
                clinical_category[index] = tmp_clinical_category
                allele_length[index] = tmp_allele_length
                partial_scores[index] = tmp_partial_scores
                print
                if not math.isnan(patients_row["Year_at_evaluation"]):
                    age_at_evaluation[index] = patients_row["Year_at_evaluation"] - patients_row["Date_of_Birth"].year
                else:
                    age_at_evaluation[index] = 2021 - patients_row["Date_of_Birth"].year
                    
    graph = nx.Graph()
    dag = nx.DiGraph()
    
    print("\n\n\nprocessing family", familyName)
    if len(personId) <2:
        print("Family ", familyName, " contains less than 2 people.")
        continue
    numberOfFamilies = numberOfFamilies + 1

    for index, row in family.iterrows():
        if row['FATHER'] != "0" or row['MOTHER'] !="0":
            graph.add_edge(index,row['FATHER'])
            graph.add_edge(index,row['MOTHER'])
            dag.add_edge(row['FATHER'],index)
            dag.add_edge(row['MOTHER'],index)
    mates_list = []
    for node in dag.nodes:
        if dag.in_degree(node) == 0:
            partners_list = []
            for children in dag.successors(node):
                for other_parent in dag.predecessors(children):
                    if other_parent != node:
                        partners_list.append(other_parent)
            are_root_of_dag = True
            for partner in partners_list:
                are_root_of_dag &= (dag.in_degree(partner) == 0)
            if not are_root_of_dag:
                mates_list.append(node)


    # mates_list = [node for node in list(nx.topological_sort(dag))[2:] if (dag.in_degree(node) == 0)]
    mates_list_negatives = []
    for node in mates_list:
        if node in patients_df.index:
            if math.isnan(fshd_score.get(node)) or not isinstance(fshd_score.get(node),float):
                mates_list_negatives.append(node)
            else:
                if not (fshd_score.get(node) == 0 and DRA.get(node) == 0):
                    mates_list_negatives.append(node)
        else:
            mates_list_negatives.append(node)
    graph.remove_nodes_from(mates_list_negatives)
    print("mates_list_negatives",mates_list_negatives)

    print("family", familyName, "contains", len(personId),"people", personId)
    peopleInFamily = 0
    for pid in personId:
        print("Creating dataset entry for person : ", pid, "with fshd score ", fshd_score.get(pid))
        if pid in mates_list_negatives:
            # print("person", pid, "is a mate. Skipping.")
            continue
        max_score_kinship = [-1,-1,-1,-1]
        max_partial_scores_per_kinship = [[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1]]
        peopleInFamily+=1
        for index, row in family.iterrows():
            if index in mates_list_negatives:
                continue
            if index != pid and (index in patients_df.index):
                person_score = fshd_score.get(index)
                if isinstance(person_score,float) and not math.isnan(person_score):
                    # length = nx.shortest_path_length(graph,pid,index)
                    length = calculateDegreeOfKinship(dag,pid,index)
                    if length < 1:
                        continue
                    index_kinship = length - 1
                    if index_kinship > 3:
                        index_kinship = 3
                    max_score_kinship[index_kinship] = max(max_score_kinship[index_kinship], person_score)
                    for muscle_group_index in range(len(max_partial_scores_per_kinship[index_kinship])):
                        tmp_partial_scores = partial_scores.get(index)
                        max_partial_scores_per_kinship[index_kinship][muscle_group_index] = max(max_partial_scores_per_kinship[index_kinship][muscle_group_index], tmp_partial_scores[muscle_group_index])
        print(pid, gender.get(pid), age_at_evaluation.get(pid), DRA.get(pid), allele_length.get(pid), fshd_score.get(pid), clinical_category.get(pid), max_score_kinship, max_partial_scores_per_kinship)
        training_df = training_df.append({'gender': gender.get(pid),'age_at_evaluation': age_at_evaluation.get(pid),'DRA': DRA.get(pid), 'allele_length': allele_length.get(pid), 
                                 'FSHD_score' : fshd_score.get(pid), 'category':  clinical_category.get(pid),
                                 'max_score_1st_kinship': max_score_kinship[0], 'max_score_2nd_kinship': max_score_kinship[1],
                                 'max_score_3rd_kinship': max_score_kinship[2], 'max_score_higher_kinship': max_score_kinship[3],
                                 'max_facial_partial_score_1st_kinship': max_partial_scores_per_kinship[0][0], 'max_facial_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][0], 'max_facial_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][0], 'max_facial_partial_score_higher_kinship': max_partial_scores_per_kinship[3][0],
                                 'max_scapular_girdle_partial_score_1st_kinship': max_partial_scores_per_kinship[0][1], 'max_scapular_girdle_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][1], 'max_scapular_girdle_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][1], 'max_scapular_girdle_partial_score_higher_kinship': max_partial_scores_per_kinship[3][1],
                                 'max_upper_limbs_partial_score_1st_kinship': max_partial_scores_per_kinship[0][2], 'max_upper_limbs_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][2], 'max_upper_limbs_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][2], 'max_upper_limbs_partial_score_higher_kinship': max_partial_scores_per_kinship[3][2],
                                 'max_legs_partial_score_1st_kinship': max_partial_scores_per_kinship[0][3], 'max_legs_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][3], 'max_legs_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][3], 'max_legs_partial_score_higher_kinship': max_partial_scores_per_kinship[3][3],
                                 'max_pelvic_girdle_partial_score_1st_kinship': max_partial_scores_per_kinship[0][4], 'max_pelvic_girdle_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][4], 'max_pelvic_girdle_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][4], 'max_pelvic_girdle_partial_score_higher_kinship': max_partial_scores_per_kinship[3][4],
                                 'max_abdomen_partial_score_1st_kinship': max_partial_scores_per_kinship[0][5], 'max_abdomen_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][5], 'max_abdomen_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][5], 'max_abdomen_partial_score_higher_kinship': max_partial_scores_per_kinship[3][5]   }, ignore_index=True)
    maxIndividualsInFamily = max(maxIndividualsInFamily, peopleInFamily)
    print(graph)
    fig = plt.figure(figsize=(16,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_facecolor('white')
    # fig.suptitle(familyName)
    # print(familyName)

    options = {
        "font_size": 18,
        # "node_size": 3000,
        "node_color": "white",
        # "node_color":"none",
        # "edgecolors": "black",
        "linewidths": 2,
        "width": 2,
        "bbox":dict(facecolor='white', edgecolor='black', boxstyle='round,pad=1'),
    }
    pos = nx.spring_layout(graph,k=1.2, seed=1234)
    nx.draw_networkx(graph, pos=pos, **options)
    fig.savefig("family_graphs/"+str(familyName)+".pdf")
    fig.clf()


print("Number of processed families: ", numberOfFamilies)

print("Number of individuals",training_df.shape[0])

print("Largest family contains", maxIndividualsInFamily)

In [ ]:

training_df.head()


### Final dataset preparation 
This is a final step before the actual training. We remove all the empty and nan values from the dataset, and make sure that numeric values are treated properly. 



In [ ]:
training_df['gender'].replace('m','M', regex=True, inplace=True)
training_df['gender'].replace('f','F', regex=True, inplace=True)
training_df['gender'].replace('F','0', regex=True, inplace=True)
training_df['gender'].replace('M','1', regex=True, inplace=True)
training_df["gender"] = pd.to_numeric(training_df["gender"])
def severity_from_score(row):
    if row['FSHD_score'] >=3 :
        return 1
    return 0


training_df["severity"] = training_df['category'].replace(["A\d","B\d", "D\d", "C\d"],[2,1,1,0],regex=True, inplace=False)
training_df["severity_binary"] = training_df['category'].replace(["A\d","B\d", "D\d", "C\d"],[1,1,1,0],regex=True, inplace=False)
training_df["severity_from_score_binary"] = training_df.apply (lambda row: severity_from_score(row), axis=1)

training_df["age_at_evaluation"] = pd.to_numeric(training_df["age_at_evaluation"])



training_df["DRA"] = pd.to_numeric(training_df["DRA"])
training_df["allele_length"] = pd.to_numeric(training_df["allele_length"])

for k in training_df.columns:
    if "score" in k:
        training_df[k] = pd.to_numeric(training_df[k])

print(training_df.category.unique())

print(training_df.severity.unique())
print(training_df.severity_binary.unique())
print("training_df.severity_from_score_binary", training_df.severity_from_score_binary.unique())
print("training_df.age_at_evaluation", training_df.age_at_evaluation.unique())

print(training_df.category.unique())

print(training_df.gender.unique())
print(training_df.FSHD_score.unique())
print(training_df.allele_length.unique())
print(training_df.max_facial_partial_score_1st_kinship.unique())
print(training_df.max_scapular_girdle_partial_score_1st_kinship.unique())
print(training_df.max_upper_limbs_partial_score_1st_kinship.unique())
print(training_df.max_legs_partial_score_1st_kinship.unique())
print(training_df.max_pelvic_girdle_partial_score_1st_kinship.unique())
print(training_df.max_abdomen_partial_score_1st_kinship.unique())

print(training_df.max_score_1st_kinship.unique())

print(training_df.columns)

print(training_df)



# Saving data to HDF



In [ ]:
training_df.to_hdf('training_data.h5', key='df', mode='w')

### Splitting data into independent and dependent variables
We want to predict the FSHD score through all other variables. 
We will then use `X` to represent all the data used to make the classification, and `Y` to represent the FSHD score that we would like to predict.

In [ ]:
training_dataset = pd.read_hdf('training_data.h5')

In [ ]:
X = training_dataset.drop(['severity','severity_binary', 'severity_from_score_binary','FSHD_score','category',
 'DRA',
 #'max_score_1st_kinship',
#  'max_score_2nd_kinship',
 #'max_score_3rd_kinship', 
 #'max_score_higher_kinship'
    ], axis = 1).copy()
X.head()
X.dtypes

In [ ]:
y=training_dataset['severity'].copy()
y_binary=training_dataset['severity_binary'].copy()
y_from_score_binary=training_dataset['severity_from_score_binary'].copy()

y_from_score_binary.head()


## Build a preliminary XGBoost model
We split the data into training and testing sets and build the model. Since data is imbalanced, we split using stratification .


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=27, stratify=y)
X_binary_train, X_binary_test, y_binary_train, y_binary_test = train_test_split(X, y_binary, random_state=27, stratify=y)



In [ ]:
# clf_xgb = xgb.XGBClassifier(objective='multi:softprob', missing=None, seed=42, num_class=3)
# clf_xgb.fit(X_train,y_train,verbose=True,early_stopping_rounds=10,eval_metric='mlogloss', eval_set=[(X_test,y_test)])
clf_xgb_binary = xgb.XGBClassifier(objective='binary:logistic', seed=42)
clf_xgb_binary.fit(X_binary_train,y_binary_train,verbose=True,early_stopping_rounds=10,eval_metric='aucpr', eval_set=[(X_binary_test,y_binary_test)])

In [ ]:
plot_confusion_matrix(clf_xgb_binary,X_binary_test,y_binary_test, normalize="true",values_format='f',display_labels=["No symptom","FSHD phenotype"])


In [ ]:
clf_xgb = xgb.XGBClassifier(objective='multi:softprob', seed=42, num_class=3)
clf_xgb.fit(X_train,y_train,verbose=True,early_stopping_rounds=10,eval_metric='mlogloss', eval_set=[(X_test,y_test)])
plot_confusion_matrix(clf_xgb,X_test,y_test, normalize="true",values_format='f')




### Hyperparameter optimization

This was executed and gave in output:
```
{'gamma': 0.25, 'learning_rate': 0.5, 'max_depth': 5, 'reg_lambda': 10.0, 'scale_pos_weight': 1}
```

In [ ]:

# param_grid = {
#     'max_depth' : [3,4,5],
#     'learning_rate': [0.1, 0.01, 0.05, 0.5, 1],
#     'gamma': [0, 0.25, 1.0],
#     'reg_lambda' : [0., 1., 10., 20., 100.],
#     'scale_pos_weight' : [1, 3, 5]
# }

# optimal_params = GridSearchCV(
#     estimator = clf_xgb_binary, 
#     param_grid = param_grid,
#     scoring = 'roc_auc', 
#     verbose = 2,
#     n_jobs=4, 
#     cv =3
# )

# optimal_params.fit(X_binary_train,y_binary_train, early_stopping_rounds=10, eval_metric="auc", eval_set=[(X_binary_test,y_binary_test)], verbose=False)
# print(optimal_params.best_params_)
# {'gamma': 0.25, 'learning_rate': 0.5, 'max_depth': 5, 'reg_lambda': 10.0, 'scale_pos_weight': 1}

Now running the optimized model

In [ ]:
clf_xgb_binary_optimized = xgb.XGBClassifier(objective='binary:logistic',gamma=0.25, learning_rate=0.5, max_depth=5, reg_lambda=10, scale_pos_weight=1, seed=42,)
clf_xgb_binary_optimized.fit(X_binary_train,y_binary_train, eval_set=[(X_binary_test,y_binary_test)])
clf_xgb_binary_optimized.save_model("fshd_binary_risk_analysis.json")
plot_confusion_matrix(clf_xgb_binary_optimized,X_binary_test,y_binary_test, normalize="true",values_format='f',display_labels=["No symptom","FSHD phenotype"])


In [ ]:
# bst = clf_xgb_binary_optimized.get_booster()
# for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
#     print("%s: " % importance_type, bst.get_score(importance_type=importance_type))

# node_params = {'shape': 'box', 'style': 'filled, rounded', 'fillcolor': '#78cbe'}
# leaf_params = {'shape': 'box', 'style': 'filled', 'fillcolor': '#e48038'}

# graph_data=xgb.to_graphviz(clf_xgb_binary_optimized,num_trees=0,size="10,10",condition_node_params=node_params,leaf_node_params=leaf_params)
# graph_data.view(filename='xgb_tree_fshd_binary')


In [ ]:
y_preds = clf_xgb_binary_optimized.predict_proba(X_binary_test)
preds = y_preds[:,1]
fpr, tpr, _ = roc_curve(y_binary_test, preds)
auc_score = auc(fpr, tpr)
plt.clf()
fig = plt.gcf()
fig.set_size_inches(12.5, 8.5)

plt.title('ROC Curve')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

# it's helpful to add a diagonal to indicate where chance 
# scores lie (i.e. just flipping a coin)
plt.plot([0,1],[0,1],'r--')

plt.xlim([0,1])
plt.ylim([0,1.005])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc='lower right')
fig.savefig('roc_binary.pdf', dpi=100)
plt.show()
plt.close()


### K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import interp
labels_predictions = {}
classifiers = {}

predictions = [y_from_score_binary,y_binary]
labels_classes = [['FSHD score $\leq 2$','FSHD score $> 2$'],['no phenotype','myopathic\nphenotype']]
labels_predictions[0] = "y_from_score_binary"
labels_predictions[1] = "y_binary"
thresholds_predictions=[0.3,0.50]
label_index = 0
for prediction in predictions:
    n_splits = 20
    clf_xgb_binary_optimized_cv = xgb.XGBClassifier(objective='binary:logistic',gamma=0.25, learning_rate=0.5, max_depth=8, reg_lambda=10, scale_pos_weight=1, seed=42,num_parallel_tree = 10, use_label_encoder =False,eval_metric ='auc',  missing=-1)
    kfold = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state=1)
    results = cross_val_score(clf_xgb_binary_optimized_cv, X, prediction, cv=kfold)
    print(results)
    tprs = []
    aucs = []
    confusion_matrix_cv_array = []
    confusion_matrix_cv_normalized_array = []
    mean_fpr = np.linspace(0, 1, 100)
    i = 0
    for fold_index, (train_index,val_index) in enumerate(kfold.split(X, prediction)):
        print('Batch {} started...'.format(fold_index))
        bst_cv = clf_xgb_binary_optimized_cv.fit(X.loc[train_index,:],prediction.loc[train_index],
                eval_set = [(X.loc[val_index,:],prediction.loc[val_index])],
                early_stopping_rounds=10,
                verbose= 200, 
                eval_metric ='auc'
                )
        probas_ = clf_xgb_binary_optimized_cv.predict_proba(X.loc[val_index,:])

        fpr, tpr, thresholds = roc_curve(prediction.loc[val_index], probas_[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        # plt.plot(fpr, tpr, lw=1, alpha=0.3,
                #  label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        confusion_matrix_cv_array.append(confusion_matrix(prediction.loc[val_index], 
                                        clf_xgb_binary_optimized_cv.predict_proba(X.loc[val_index,:])[:,1]>=thresholds_predictions[label_index], normalize=None))
        confusion_matrix_cv_normalized_array.append(confusion_matrix(prediction.loc[val_index], 
                                        clf_xgb_binary_optimized_cv.predict_proba(X.loc[val_index,:])[:,1]>=thresholds_predictions[label_index], normalize="true"))
        i += 1

    # y_preds = clf_xgb_binary_optimized_cv.predict_proba(X_binary_test)
    # preds = y_preds[:,1]
    # fpr, tpr, _ = roc_curve(prediction_test, preds)
    # auc_score = auc(fpr, tpr)
    # plt.clf()
    # fig = plt.gcf()
    # fig.set_size_inches(12.5, 8.5)

    # plt.title('ROC Curve')
    # plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

    # # it's helpful to add a diagonal to indicate where chance 
    # # scores lie (i.e. just flipping a coin)
    # plt.plot([0,1],[0,1],'r--')

    # plt.xlim([0,1])
    # plt.ylim([0,1.005])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')

    # plt.legend(loc='lower right')
    # plt.show()
    # fig.savefig('roc.pdf', dpi=100)
    print(confusion_matrix_cv_array)


    cm_final  = [[0., 0.],[0., 0.]]

    cm_final_normalized  = [[0., 0.],[0., 0.]]

    for c in confusion_matrix_cv_array:
        cm_final = cm_final + c


    for c in confusion_matrix_cv_normalized_array:
        cm_final_normalized = cm_final_normalized + c

    cm_final_normalized = cm_final_normalized/n_splits


    tn, fp, fn, tp = cm_final.ravel()

    y_line_threshold = tp/(tp+fn)
    x_line_threshold = fp/(fp+tn)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.figure(figsize=(10,10))

    plt.clf()
    fig = plt.gcf()
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

    plt.plot([x_line_threshold,x_line_threshold], [0,1], linestyle='dotted', lw=2, color='black',
            label="Threshold: "+str(thresholds_predictions[label_index]*100)+"%", alpha=.8)


    plt.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('False Positive Rate',fontsize=18)
    plt.ylabel('True Positive Rate',fontsize=18)
    plt.title('Cross-Validation ROC',fontsize=18)
    plt.legend(loc="lower right", prop={'size': 15})
    fig = plt.gcf()
    plt.show()
    fig.savefig('roc_'+ labels_predictions[label_index]+'.pdf', dpi=100)
    model_fname = "fshd_binary_risk_analysis_cv_" + labels_predictions[label_index]+".json"
    clf_xgb_binary_optimized_cv.save_model(model_fname)
    plt.cla()
    plt.clf()
    fig.clf()

    

    xgb.plot_importance(clf_xgb_binary_optimized_cv, max_num_features=10)
    fig = plt.gcf()
    fig.set_dpi(100)
    fig.set_size_inches(10,4)
    plt.tight_layout()


    fig.savefig('features_importance_'+ labels_predictions[label_index]+'.pdf', dpi=100)
    plt.show()

    
    from sklearn.metrics import ConfusionMatrixDisplay
    plt.clf()
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_final)

    # NOTE: Fill all variables here with default values of the plot_confusion_matrix
    # disp = disp.plot()
    disp.display_labels = labels_classes[label_index]
    disp.plot(values_format = '.5g')
    plt.tight_layout()
    fig = plt.gcf()
    fig.savefig("confusion_matrix_"+labels_predictions[label_index]+".pdf", dpi=100)
    plt.show()

    
    plt.cla()
    plt.clf()
    fig.clf()

    disp_normalized = ConfusionMatrixDisplay(confusion_matrix=cm_final_normalized)
    disp_normalized.display_labels = labels_classes[label_index]
    disp_normalized.plot(values_format = '.5g')
    plt.tight_layout()
    fig = plt.gcf()

    

    fig.savefig("confusion_matrix_normalized_"+labels_predictions[label_index]+".pdf", dpi=100)
    plt.show()

    bst = clf_xgb_binary_optimized_cv.get_booster()
    for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
        print("%s: " % importance_type, bst.get_score(importance_type=importance_type))

    node_params = {'shape': 'box', 'style': 'filled, rounded', 'fillcolor': '#78cbe'}
    leaf_params = {'shape': 'box', 'style': 'filled', 'fillcolor': '#e48038'}

    graph_data=xgb.to_graphviz(clf_xgb_binary_optimized_cv,num_trees=0,size="10,10",condition_node_params=node_params,leaf_node_params=leaf_params)
    graph_data.view(filename='xgb_tree_fshd_binary_'+ labels_predictions[label_index])
    classifiers[labels_predictions[label_index]] = clf_xgb_binary_optimized_cv

    label_index+=1



### Evaluation of a new family 

Store the haplopainter csv file in the `input` folder of the new family. 

In [ ]:
print(pd.DataFrame(columns=X.columns))

In [ ]:
import re

graph = nx.Graph()

dag = nx.DiGraph()

def generatePredictionFromPersonId(target_personId="-1"):


    input_family_df = []

    print("Searching for family trees in the input folder in csv format with utf8 extension")
    for root, dirs, files in os.walk("input/"):
        for file in files:
            if file.endswith(".utf8"):
                filePath = os.path.join(root, file)
                fix_family_file(filePath)
                input_family_df.append(pd.read_csv(filePath, header=None, usecols=[0,1,2,3], names=['FAMILY', 'PERSON','FATHER','MOTHER'], delimiter='\t', index_col=1))
                print("found family tree: ", filePath)
    print(len(input_family_df)," family trees found")
    if len(input_family_df) == 0:
        sys.exit("No family found in folder.")
    if len(input_family_df) > 1:
        sys.exit("More than one family found in folder. Please remove the unnecessary families from input folder.")

    if target_personId == "-1":
        target_personId = input("Insert the person id in the tree: ")


    new_prediction_df = pd.DataFrame(columns=X.columns)
    target_DRA = 0
    target_allele_length = 0
    target_gender = "m"
    print("Checking whether ", target_personId, "is already in registry")
    if target_personId in patients_df.index:
        target_DRA = patients_df.get("DRA_(0=no;1=si)").get(target_personId)
        target_allele_length = patients_df.get("allele_size_in_kb").get(target_personId)
        target_gender = patients_df.get("Sex").get(target_personId)
        print("Person ", target_personId, " found in registry")
    else:
        print(target_personId, "is not in registry (or its data are incomplete). You need to feed data manually")
        while True:
            target_DRA = input("Insert 1 if person has DRA, or 0 if not: ")
            if target_DRA not in ("0", "1"):
                print("Not an appropriate choice.")
            else:
                break

        while True:
            target_allele_length = input("Insert the allele length in kb: ")
            if isinstance(target_allele_length,int):
                print("Please insert an integer number.")
            else:
                break
        
        while True:
            target_gender = input("Insert the gender (f, m): ")
            if target_gender not in ("m", "f", "M", "F"):
                print("Not an appropriate choice.")
            else:
                break
    print("DRA: ", target_DRA)
    print("Allele length: ", target_allele_length,"kb")
    print("Gender: ", target_gender)


    for df in input_family_df: 
        family = df[['FAMILY','FATHER','MOTHER']]
        familyName = ""
        personId=[]
        personId.append(target_personId)
        fshd_score ={}
        gender = {}
        DRA = {}
        clinical_category = {}
        allele_length = {}
        partial_scores = {}
        for index, row in family.iterrows():
            familyName = row['FAMILY']
            if '/' in index and index in patients_df.index:
                tmp_score = patients_df.get("FSHD_score").get(index)
                tmp_allele_length = patients_df.get("allele_size_in_kb").get(index)
                tmp_clinical_category = patients_df.get("clinical_category").get(index)
                tmp_partial_scores = [patients_df.get("FACIALWEAKNESS").get(index),
                                    patients_df.get("SCAPGIRDLEINVOLV").get(index),
                                    patients_df.get("UPPERLIMBSINVOLV").get(index),
                                    patients_df.get("LEGSINVOLVEMENT").get(index),
                                    patients_df.get("PELVGIRDLEINVOLV").get(index),
                                    patients_df.get("ABDMUSCLEINVOLV").get(index)]
                if index != target_personId and isinstance(tmp_score,float) and not math.isnan(tmp_score) and not math.isnan(tmp_allele_length) and tmp_clinical_category != "Z":
                    # personId.append(index)
                    fshd_score[index] = tmp_score
                    gender[index] = patients_df.get("Sex").get(index)
                    DRA[index] = patients_df.get("DRA_(0=no;1=si)").get(index)
                    clinical_category[index] = tmp_clinical_category
                    allele_length[index] = tmp_allele_length
                    partial_scores[index] = tmp_partial_scores



        for index, row in family.iterrows():
            if row['FATHER'] != "0" or row['MOTHER'] !="0":
                graph.add_edge(index,row['FATHER'])
                graph.add_edge(index,row['MOTHER'])
                dag.add_edge(row['FATHER'],index)
                dag.add_edge(row['MOTHER'],index)
        mates_list = []
        for node in dag.nodes:
            if dag.in_degree(node) == 0:
                partners_list = []
                for children in dag.successors(node):
                    for other_parent in dag.predecessors(children):
                        if other_parent != node:
                            partners_list.append(other_parent)
                are_root_of_dag = True
                for partner in partners_list:
                    are_root_of_dag &= (dag.in_degree(partner) == 0)
                if not are_root_of_dag:
                    mates_list.append(node)


        # mates_list = [node for node in list(nx.topological_sort(dag))[2:] if (dag.in_degree(node) == 0)]
        mates_list_negatives = []
        for node in mates_list:
            if node in patients_df.index:
                if math.isnan(fshd_score.get(node)) or not isinstance(fshd_score.get(node),float):
                    mates_list_negatives.append(node)
                else:
                    if not (fshd_score.get(node) == 0 and DRA.get(node) == 0):
                        mates_list_negatives.append(node)
            else:
                mates_list_negatives.append(node)
        graph.remove_nodes_from(mates_list_negatives)

        for pid in personId:
            print("Creating dataset entry for person : ", pid, "with fshd score ", fshd_score.get(pid))
            if pid in mates_list_negatives:
                # print("person", pid, "is a mate. Skipping.")
                continue
            # print(tabulate(patients_df.loc[[pid]],headers='keys', tablefmt='psql'))
            max_score_kinship = [-1,-1,-1,-1]
            max_partial_scores_per_kinship = [[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1]]
            for index, row in family.iterrows():
                if index in mates_list_negatives:
                    continue
                if index != pid and (index in patients_df.index):
                    person_score = fshd_score.get(index)
                    if isinstance(person_score,float) and not math.isnan(person_score):
                        # length = nx.shortest_path_length(graph,pid,index)
                        length = calculateDegreeOfKinship(dag,pid,index)
                        if length < 1:
                            continue
                        index_kinship = length - 1
                        if index_kinship > 3:
                            index_kinship = 3
                        max_score_kinship[index_kinship] = max(max_score_kinship[index_kinship], person_score)
                        for muscle_group_index in range(len(max_partial_scores_per_kinship[index_kinship])):
                            tmp_partial_scores = partial_scores.get(index)
                            max_partial_scores_per_kinship[index_kinship][muscle_group_index] = max(max_partial_scores_per_kinship[index_kinship][muscle_group_index], tmp_partial_scores[muscle_group_index])


            new_prediction_df = new_prediction_df.append({'gender': target_gender,
                                    # 'DRA': target_DRA, 
                                    'allele_length': target_allele_length,
                                    # 'age_at_evaluation': age_at_evaluation[pid],
                                    'max_score_1st_kinship': max_score_kinship[0], 'max_score_2nd_kinship': max_score_kinship[1], 'max_score_3rd_kinship': max_score_kinship[2], 'max_score_higher_kinship': max_score_kinship[3],
                                    
                                    'max_facial_partial_score_1st_kinship': max_partial_scores_per_kinship[0][0], 'max_facial_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][0], 'max_facial_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][0], 'max_facial_partial_score_higher_kinship': max_partial_scores_per_kinship[3][0],
                                    'max_scapular_girdle_partial_score_1st_kinship': max_partial_scores_per_kinship[0][1], 'max_scapular_girdle_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][1], 'max_scapular_girdle_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][1], 'max_scapular_girdle_partial_score_higher_kinship': max_partial_scores_per_kinship[3][1],
                                    'max_upper_limbs_partial_score_1st_kinship': max_partial_scores_per_kinship[0][2], 'max_upper_limbs_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][2], 'max_upper_limbs_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][2], 'max_upper_limbs_partial_score_higher_kinship': max_partial_scores_per_kinship[3][2],
                                    'max_legs_partial_score_1st_kinship': max_partial_scores_per_kinship[0][3], 'max_legs_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][3], 'max_legs_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][3], 'max_legs_partial_score_higher_kinship': max_partial_scores_per_kinship[3][3],
                                    'max_pelvic_girdle_partial_score_1st_kinship': max_partial_scores_per_kinship[0][4], 'max_pelvic_girdle_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][4], 'max_pelvic_girdle_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][4], 'max_pelvic_girdle_partial_score_higher_kinship': max_partial_scores_per_kinship[3][4],
                                    'max_abdomen_partial_score_1st_kinship': max_partial_scores_per_kinship[0][5], 'max_abdomen_partial_score_2nd_kinship': max_partial_scores_per_kinship[1][5], 'max_abdomen_partial_score_3rd_kinship': max_partial_scores_per_kinship[2][5], 'max_abdomen_partial_score_higher_kinship': max_partial_scores_per_kinship[3][5]
                                    
            },ignore_index=True)

        # print(graph.graph)
        # fig = plt.figure(figsize=(16,10))
        # ax = fig.add_subplot(1, 1, 1)
        # ax.set_facecolor('white')
        # fig.suptitle(familyName)
        # # print(familyName)

        # options = {
        #     "font_size": 18,
        #     "node_size": 2000,
        #     "node_color": "white",
            
        #     "edgecolors": "black",
        #     "linewidths": 2,
        #     "width": 2,
        # }
        # nx.draw_networkx(graph, pos=nx.spring_layout(graph,k=1.2), **options)


    new_prediction_df['gender'].replace(['m','M'],[1,1], regex=True, inplace=True)
    new_prediction_df['gender'].replace(['f','F'],[0,0], regex=True, inplace=True)
    for k in new_prediction_df.columns:
        new_prediction_df[k] = pd.to_numeric(new_prediction_df[k])
    return new_prediction_df.copy(),  target_personId, target_allele_length


In [ ]:
query, target_personId, target_allele_length = generatePredictionFromPersonId()

In [ ]:
print(graph.nodes)
print(dag.nodes)


fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('white')
fig.suptitle(familyName)
# print(familyName)

options = {
    "font_size": 18,
    # "node_size": 3000,
    "node_color": "white",
    # "node_color":"none",
    # "edgecolors": "black",
    # "linewidths": 2,
    "width": 2,
    "bbox":dict(facecolor="white",alpha=0.2,edgecolor='black', boxstyle='round,pad=1'),
    "arrows":True,
    "arrowsize":10, 
    "arrowstyle":'-|>',
}
pos = nx.spring_layout(dag,k=1.2, seed=1234)
nx.draw_networkx(dag, pos=pos, **options)


In [ ]:
best_iteration = classifiers['y_binary'].get_booster().best_iteration
print("computing for ",target_personId)
avg_result = 0
ages = [30]
for age in ages:
    query.iloc[0,query.columns.get_loc('age_at_evaluation')] = age
    result = classifiers['y_binary'].predict_proba(query,iteration_range=(0, best_iteration))[0][1]
    print(result*100,"%")
    avg_result += result
    # print(query)

avg_result /= len(ages)
print("The probability that ", target_personId, "with allele length of", target_allele_length, "kb \nwill show the phenotype is ", "{:.1f}".format(avg_result*100),"%")
if avg_result > 0.5:
    print("Subject classified as myopathic phenotype")
else:
    print("Subject classified as non-myopathic phenotype")
if target_personId in patients_df.index:
    print(patients_df.loc[target_personId])

In [ ]:
print(query)

In [ ]:
for person in dag.nodes:
    if "/" in person and person in patients_df.index:
        query, target_personId, target_allele_length = generatePredictionFromPersonId(person)
        best_iteration = classifiers['y_binary'].get_booster().best_iteration
        print("computing for ",target_personId)
        avg_result = 0
        ages = [30]
        for age in ages:
            query.iloc[0,query.columns.get_loc('age_at_evaluation')] = age
            result = classifiers['y_binary'].predict_proba(query,iteration_range=(0, best_iteration))[0][1]
            # print(result*100,"%")
            avg_result += result
            # with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            #     print(query)

        avg_result /= len(ages)
        print("The probability that ", target_personId, "with allele length of", target_allele_length, "kb \nwill show the phenotype is ", "{:.1f}".format(avg_result*100),"%")
        if avg_result > 0.5:
            print("Subject classified as myopathic phenotype")
        else:
            print("Subject classified as non-myopathic phenotype")